In [13]:
import re
from PIL import Image, ImageChops
import os
import matplotlib.pyplot as plt
import cv2

def extract_s_number(filename):
    # Utilizza un'espressione regolare per estrarre il valore S0-S59 alla fine del nome del file
    match = re.search(r'_S(\d+)\.png$', filename)
    if match:
        return int(match.group(1))
    return -1  # Restituisce -1 se non trova un valore S0-S59 alla fine del nome

def custom_sort_by_s_number(file_list):
    # Funzione di ordinamento personalizzata basata su S0-S59
    return sorted(file_list, key=extract_s_number)

def categorize_images(folder_path):
    mri_images = []
    prostate_images = []
    target_images = []

    # Itera su tutti i file nella cartella
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Verifica se è un file e se ha estensione .png
        if os.path.isfile(file_path) and filename.endswith('.png'):
            # Dividi le immagini nelle tre tipologie
            if 'mri' in filename.lower():
                mri_images.append(file_path)
            elif 'prostate' in filename.lower():
                prostate_images.append(file_path)
            elif 'target' in filename.lower():
                target_images.append(file_path)
                #print(file_path)
    #print(len(target_images))
    mri_images_sorted = custom_sort_by_s_number(mri_images)
    prostate_images_sorted = custom_sort_by_s_number(prostate_images)

    #facciamo questo doppio sort per i target, perchè possono esistere più target per una singola prostata, e quindi 60 immagini per target. 
    #perciò dividiamo prima per target1, target2 ecc e poi facciamo il sort per il valore s0,s1,s2 di ogni singolo target    print(len(target_images))
    target_images_sorted1 = sorted(target_images)

    target_images_sorted_final = []
    num_imag_mri = len(mri_images_sorted)

    for i in range(0, int(len(target_images_sorted1)/num_imag_mri)):
         target_images_sorted_final.append(custom_sort_by_s_number(target_images_sorted1[num_imag_mri*i:num_imag_mri+num_imag_mri*i]))
    # for mri_image in mri_images_sorted:
    # #     image = Image.open(mri_image)
    # #     #plt.imshow(image)
    #     plt.title(mri_image)
    #     plt.show()
    return mri_images_sorted, prostate_images_sorted, target_images_sorted_final

def is_mask_present(mask_path):
    # Carica l'immagine della maschera
    mask = Image.open(mask_path)

    # Assicurati che la maschera sia in scala di grigi
    if mask.mode != "L":
        mask = mask.convert("L")

    # Converte la maschera in una matrice di valori binari (0 o 255)
    mask = mask.point(lambda x: 0 if x < 128 else 255)

    # Controlla se la maschera ha almeno un pixel bianco (cioè, esiste una maschera)
    mask_present = any(pixel == 255 for pixel in mask.getdata())

    return mask_present

def apply_mask(image_path, mask_path):
    try:
        # Carica le immagini
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # Assicurati che le dimensioni delle due immagini siano compatibili
        if image.size != mask.size:
            raise ValueError("Le dimensioni dell'immagine e della maschera non corrispondono")

        # Converte la maschera in scala di grigi se non lo è già
        if mask.mode != "L":
            mask = mask.convert("L")           

        # Converte la maschera in una matrice di valori binari (0 o 255)
        mask = mask.point(lambda x: 0 if x < 128 else 255)

        # Applica la maschera all'immagine originale per mantenere le caratteristiche di colore
        image = Image.composite(image, Image.new("RGB", image.size, color="black"), mask)


        # # Find bounding box of the mask
        # bbox = mask.getbbox()
        # if bbox is None:
        #     raise ValueError("La maschera non contiene alcun pixel attivo")
        # # Calculate the desired zoom factor (e.g., 0.8 for 80% zoom)
        # zoom_factor = 0.4

        # # Calculate the dimensions of the cropped area based on the zoom factor
        # crop_width = (1 - zoom_factor) * (bbox[2] - bbox[0])
        # crop_height = (1 - zoom_factor) * (bbox[3] - bbox[1])

        # # Calculate the new crop margin based on the zoomed dimensions
        # crop_margin = (crop_width / 2, crop_height / 2)

        # # Calculate crop margin as the dimensions of the mask
        # #crop_margin = (bbox[2] - bbox[0], bbox[3] - bbox[1])

        # # Add crop margin to the bounding box
        # crop_bbox = (
        #     bbox[0] - crop_margin[0],
        #     bbox[1] - crop_margin[1],
        #     bbox[2] + crop_margin[0],
        #     bbox[3] + crop_margin[1],
        # )

        # # Crop the image with the expanded bounding box
        # cropped_image = image.crop(crop_bbox)
        # Resize the cropped image to 256x256
        #resized_image = cropped_image.resize((256, 256))
        #image = image.resize((256, 256))
        return image

    except Exception as e:
        print("Si è verificato un errore:", str(e))

def apply_inverse_mask(image, mask_path):
    try:
        # Carica le immagini
        #image = Image.open(image_path)
        mask = Image.open(mask_path)

        # Assicurati che le dimensioni delle due immagini siano compatibili
        if image.size != mask.size:
            raise ValueError("Le dimensioni dell'immagine e della maschera non corrispondono")

        # Converte la maschera in scala di grigi se non lo è già
        if mask.mode != "L":
            mask = mask.convert("L")

        # Converte la maschera in una matrice di valori binari (0 o 255)
        mask = mask.point(lambda x: 0 if x < 128 else 255)

        # Inverti la maschera (0 diventa 255 e viceversa)
        inverse_mask = ImageChops.invert(mask)

        # Applica la maschera negata all'immagine originale per eliminare le regioni coperte dalla maschera
        image = Image.composite(image, Image.new("RGB", image.size, color="black"), inverse_mask)

        # ritorna l'immagine risultante
        return image

    except Exception as e:
        print("Si è verificato un errore inverse:", str(e))

def apply_white_background(image_path):
    # Read the image using OpenCV (in BGR format)
    image = cv2.imread(image_path)

    # Convert the image to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a mask for the background (assuming the background is pure black)
    background_mask = (image_rgb == [0, 0, 0]).all(axis=-1)

    # Set the background to white
    image_rgb[background_mask] = [255, 255, 255]

    # Convert the image back to BGR format
    result_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Save the result with a white background
    cv2.imwrite(image_path, result_image)


def prostata_is_big(image_path):
    #image_path = 'ciaprostatex/mri/train/paziente1/Dato1/Prostate1_1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221_S12.png'
    ret = False
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Trova i contorni della maschera
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Trova il contorno più grande (presumibilmente la maschera)
    largest_contour = max(contours, key=cv2.contourArea)

    # Calcola il rettangolo del contorno più grande
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Stampa le dimensioni della maschera
    #print("Dimensioni della maschera (larghezza x altezza):", w, "x", h)
    if w>=40 and h >=40: 
        ret = True

    return ret

if __name__ == "__main__":
    # Specifica il percorso della cartella contenente le immagini
    folder_path = 'ciaprostatex/mri/train/'
    count=0
    count_good_test =0
    count_tumor_mask = 0
    first = False
    for filename in os.listdir(folder_path):
        
        filename = folder_path + filename + "/Dato1/"
        print(filename)
        if 'paziente' in filename: 
            mri_images, prostate_images, target_images = categorize_images(filename)
            #print(len(mri_images))
            # Assicurati che le liste abbiano la stessa lunghezza

            #fare un altro controllo se i target_image hanno tutti la stessa lunghezza TODO
            if len(mri_images) == len(prostate_images):
                # Accoppia le due liste
                mri_prostate_images = zip(mri_images, prostate_images)

                # qui l'ide è che dato che non possiamo sapere quanti target ci sono, creiamo la tupla sui valori conosciuti. 
                # e i target in base a quanti sono, facciamo un ciclo a parte in cui iteriamo sulle singole immagini da sottrarre all'immagine da salvare
                for i, (mri_image_path, prostate_image_path) in enumerate(mri_prostate_images):
                    #mri_image_path, prostate_image_path = mri_prostate_path
                    # Puoi lavorare con mri_image_path, prostate_image_path e target_path qui

                    if is_mask_present(prostate_image_path):
                        if prostata_is_big(prostate_image_path):
                            mask_mri = apply_mask(mri_image_path, prostate_image_path)
                            thereismask = False
                            mask_mri_inverse = None
                            # Qui potrebbero esserci più target per una stessa prostata.
                            for target_image_path in target_images:
                                if is_mask_present(target_image_path[i]):
                                    #print(target_image_path[i])
                                    if count >= 200 :
                                        output_image_mask_path = f"new_dataset/ground_truth/tumor/{count_tumor_mask}.png"
                                        output_image_tumor_path = f"new_dataset/test/tumor/{count_tumor_mask}.png"
                                        print(prostate_image_path)
                                        mask_mri.save(output_image_tumor_path)
                                        apply_white_background(output_image_tumor_path)
                                        
                                        image_target1=Image.open(target_image_path[i])
                                        #resized_image_target1 = image_target1.resize((256, 256))
                                        image_target1.save(output_image_mask_path)
                                        count_tumor_mask += 1
                                    if mask_mri_inverse is not None:
                                        mask_mri_inverse = apply_inverse_mask(mask_mri_inverse, target_image_path[i])
                                    else :
                                        mask_mri_inverse = apply_inverse_mask(mask_mri, target_image_path[i])
                                    thereismask = True

                            if count <= 200: 
                                # if count >= 100 :
                                #     output_image_path = f"new_dataset/test/good/{count_good_test}.png"
                                #     count_good_test += 1
                                # else :
                                output_image_path = f"new_dataset/train/good/{count}.png"

                                if thereismask == True and mask_mri_inverse != None:
                                    mask_mri_inverse.save(output_image_path)
                                else:
                                    mask_mri.save(output_image_path)
                                apply_white_background(output_image_path)                   
                            count=count+1
                            if count == 300 :
                                break

        if count == 300 :
            break


ciaprostatex/mri/train/paziente805/Dato1/
ciaprostatex/mri/train/paziente432/Dato1/
ciaprostatex/mri/train/paziente646/Dato1/
ciaprostatex/mri/train/paziente409/Dato1/
ciaprostatex/mri/train/paziente318/Dato1/
ciaprostatex/mri/train/paziente525/Dato1/
ciaprostatex/mri/train/paziente280/Dato1/
ciaprostatex/mri/train/paziente1134/Dato1/
ciaprostatex/mri/train/paziente226/Dato1/
ciaprostatex/mri/train/paziente822/Dato1/
ciaprostatex/mri/train/paziente412/Dato1/
ciaprostatex/mri/train/paziente2/Dato1/
ciaprostatex/mri/train/paziente2/Dato1/Prostate1_1.3.6.1.4.1.14519.5.2.1.186749128823666050588710146976747922803_S30.png
ciaprostatex/mri/train/paziente2/Dato1/Prostate1_1.3.6.1.4.1.14519.5.2.1.186749128823666050588710146976747922803_S31.png
ciaprostatex/mri/train/paziente2/Dato1/Prostate1_1.3.6.1.4.1.14519.5.2.1.186749128823666050588710146976747922803_S32.png
ciaprostatex/mri/train/paziente2/Dato1/Prostate1_1.3.6.1.4.1.14519.5.2.1.186749128823666050588710146976747922803_S33.png
ciaprostatex/

In [17]:
path = "ciaprostatex/mri/train/paziente1026/Dato1/Target1_1.3.6.1.4.1.14519.5.2.1.128376015188648403823446726950651318546_S36.png"
ret = False
image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

# Trova i contorni della maschera
contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Trova il contorno più grande (presumibilmente la maschera)
largest_contour = max(contours, key=cv2.contourArea)

# Calcola il rettangolo del contorno più grande
x, y, w, h = cv2.boundingRect(largest_contour)

# Stampa le dimensioni della maschera
#print("Dimensioni della maschera (larghezza x altezza):", w, "x", h)
if w>=40 and h >=40: 
    ret = True

print(w,h)

28 34
